# 用 DataRobot 分析 Hubble2 自動標註 Component

In [1]:
%run ../../forth.py

reDef unknown
reDef -->
p e f o r t h    v1.26
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.



In [2]:
import datarobot as dr 
import pandas as pd
import numpy as np
import math

In [16]:
dr.Client(token=peforth.execute('api_token').pop(), endpoint=peforth.execute('endpoint').pop())

In [17]:
# 用片段 project name 找回 project
%f dropall dr :>~ Project.list(search_params={'project_name':'hubble2 component'})
project = peforth.pop()[0]
%f project -->

project --> Project(hubble2 component) (<class 'datarobot.models.project.Project'>)


In [ ]:
project.get_featurelists()

In [ ]:
featurelist = project.get_featurelists()[0]
%f featurelist :> name -->
%f featurelist :> features -->

## hubble2 component 1

看 owner 可以知道 component。如果 feature list 用 recommended DR reduced Features M6 則從 feature impact 看來，最重要的就是 owner 然後是 subject 

In [ ]:
# 精挑 feature list 跑跑看
featurelist = project.create_featurelist(
    'hubble2 component 1', 
    ['Customer', 'Subject', 'Description', 'Creator', 'owner', 'RootCause', 'CustomerPhase'])

In [ ]:
%f featurelist -->

In [ ]:
project.set_target(  
    target='GeneralComponentType',
    worker_count = '-1', 
    mode=dr.AUTOPILOT_MODE.QUICK,
    featurelist_id=featurelist.id
)
project.wait_for_autopilot()  # Wait for autopilot to complete


In [3]:
# 2021/07/19 15:30  read the data  
df = pd.read_excel(r"c:\Users\8304018\Documents\GitHub\examples-for-data-scientists\Playground\hubble2\hubble2-2017-2019.xlsx");
df

C:\Users\8304018\AppData\Local\Continuum\anaconda3\envs\peforthDevelop\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  


,Customer,bugid,ProjectName,Creator,OpenDate,Subject,Severity,owner,Status,Description,...,CustomerPhase,CloseDate,LastUpdate,GeneralComponentType,GeneralSeverity,GeneralStatus,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,Annie,169608,Crux_WL,Potato_Chen@wistron.com,2018-12-14 21:23:00,"QT, C phase, Sound is small when set intenal s...",1,Ivan_Tseng@wistron.com,Closed,"QT, C phase, Sound is small when set intenal s...",...,NaN,2018-12-21 16:20:57,2018-12-21 16:19:56,SW_Driver,1,Closed,NaN,NaN,NaN,NaN
1,Annie,170860,Ghibli_CFS,Jianyong_Bai@wistron.com,2019-01-15 13:11:00,"QT, Ghibil,GhibliC,Win10.17134.Pro, Final Fant...",1,Rossi_Lin@wistron.com,Closed,"QT, Ghibil,GhibliC,Win10.17134.Pro, Final Fant...",...,NaN,2019-01-29 15:44:38,2019-04-15 16:07:00,HW,1,Closed,NaN,NaN,NaN,NaN
2,Annie,167140,vCopperbox,Vicky_Kang@wistron.com,2018-09-11 15:40:00,[2DP] ML3(Win10) test result is fail.,3,Ingrid_Weng@wistron.com,Defer,[2DP] ML3(Win10) test result is fail.,...,NaN,1900-01-01 00:00:00,2018-10-01 14:34:56,SW_Driver,3,Closed,NaN,NaN,NaN,NaN
3,Annie,144950,Flavia,Apache_Lai@wistron.com,2017-02-21 13:53:00,The Screen brightness bar will show in Power O...,3,Newman_Chuang@wistron.com,Closed,The Screen brightness bar will show in Power O...,...,NaN,2017-05-12 11:19:52,2017-05-12 11:18:56,Others,3,Closed,NaN,NaN,NaN,NaN
4,Annie,164917,vGore,Xinlin_Zhong@wistron.com,2018-06-20 14:43:00,ML2 test result will shows Fail under DCHUAsse...,3,Jerry_Kuo@wistron.com,Closed,ML2 test result will shows Fail under DCHUAsse...,...,NaN,2018-07-18 11:20:14,2018-07-18 11:19:56,SW_Driver,3,Closed,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79548,UTS,BA6250,PIQ47L-VCOPPERBOX2,Tina S Lin,2019-11-12 17:03:35,Q470:Q470 is no PD-In connect pin.,1,Jimb Chang,Open,Procedure:<BR>1.Q470 is no PD-In connect pin.<...,...,NaN,1900-01-01 00:00:00,2019-11-12 17:03:35,HW,1,Open,NaN,NaN,NaN,NaN
79549,UTS,BA6705,PIQ47L-VCOPPERBOX2,Tina S Lin,2019-12-03 17:54:16,H470/Q470: RAID1 volume cann't be detect on OS...,2,Maverick Lu,Open,Procedure:<BR>1.Boot to BIOS.<BR>2.Create RAID...,...,NaN,1900-01-01 00:00:00,2019-12-03 17:54:16,Others,2,Open,NaN,NaN,NaN,NaN
79550,UTS,AV7931,MA18-LTE,Alex CW Huang,2017-02-20 10:41:01,"[TiMi] MA18_LTE, SDV : Bezel rubber 左右侧有毛边.(FR...",1,Leo TC Chen/WHQ/Wistron,Verify,"<SPAN style=""FONT-FAMILY: Calibri; COLOR: #002...",...,NaN,1900-01-01 00:00:00,2017-06-23 13:15:12,Others,1,Verify,NaN,NaN,NaN,NaN
79551,UTS,BA2136,GHIBLI_CFS,Carlor Chen,2018-12-18 11:02:21,"ENG1, Win10.17134.Pro, SKUC, NV Control panel ...",2,Kobe Kao,Open,a. Attach Acer dock to system<BR>b. Set HDMI/D...,...,NaN,1900-01-01 00:00:00,2018-12-18 11:02:21,SW,2,Open,NaN,NaN,NaN,NaN


# Annie

雖然只有 19% 正確。考慮 training data 只有 7/959 屬 Annie 似乎表示 AI 有學到一些 common insights.

In [9]:
# 用 df.query 篩選 dataframe 例如 test_data.query('300 <= Days') 
df_annie = df.query('Customer=="Annie"')

In [10]:
# 查看 shape 是 Annie 無誤
df_annie.shape

(4504, 22)

In [13]:
# 取少量來 predict 看看
annie_100 = df_annie.sample(100)
%f annie_100 :> shape -->

annie_100 :> shape --> (100, 22) (<class 'tuple'>)


In [ ]:
models = project.get_models()

In [22]:
%f project -->
%f models -->
%f cr models :> [0] --> # 這隻的 feature impact 看起來最合理，主要是根據 description 與 subject.

project --> Project(hubble2 component) (<class 'datarobot.models.project.Project'>)
models --> [Model('Keras Slim Residual Neural Network Classifier using Training Schedule (1 Layer: 64 Units)'), Model('AVG Blender'), Model('Regularized Logistic Regression (L2)'), Model('Regularized Logistic Regression (L2)'), Model('Regularized Logistic Regression (L2)'), Model('Regularized Logistic Regression (L2)'), Model('eXtreme Gradient Boosted Trees Classifier'), Model('Light Gradient Boosted Trees Classifier with Early Stopping (SoftMax Loss) (16 leaves)'), Model('RandomForest Classifier (Gini)'), Model('TensorFlow Neural Network Classifier'), Model('ExtraTrees Classifier (Gini)')] (<class 'list'>)

cr models :> [0] --> Model('Keras Slim Residual Neural Network Classifier using Training Schedule (1 Layer: 64 Units)') (<class 'datarobot.models.model.Model'>)
這隻的 feature impact 看起來最合理，主要是根據 description 與 subject.



In [23]:
model = models[0]
%f model -->

model --> Model('Keras Slim Residual Neural Network Classifier using Training Schedule (1 Layer: 64 Units)') (<class 'datarobot.models.model.Model'>)


Before actually requesting predictions, you should upload the dataset you wish to predict via Project.upload_dataset. Previously uploaded datasets can be seen under Project.get_datasets. When uploading the dataset you can provide the path to a local file, a file object, raw file content, a pandas.DataFrame object, or the url to a publicly available dataset.

In [26]:
dataset = project.upload_dataset(annie_100)

In [30]:
%f dataset --> # 他主動幫忙取了個名字 'predict.csv'
%f project :> get_datasets() -->

dataset --> PredictionDataset('predict.csv') (<class 'datarobot.models.prediction_dataset.PredictionDataset'>)
他主動幫忙取了個名字 'predict.csv'

project :> get_datasets() --> [PredictionDataset('predict.csv')] (<class 'list'>)


In [32]:
predict_job = model.request_predictions(dataset.id)
predict_job.wait_for_completion()

In [33]:
%f predict_job -->

predict_job --> PredictJob(Model('60f4e46d12f5b66853b0eb3a'), status=COMPLETED) (<class 'datarobot.models.predict_job.PredictJob'>)


### prediction 做好了，用 UI 去 download excel 比較好，會沿用上面設定好的欄位。

該 excel 需要一點整理，predict 結果都是分數要已最高值整理成結論：

    Q2 =MID(INDEX($G$1:$P$1,0,MATCH(MAX($G2:$P2),$G2:$P2,0)),12,100)
	R2 =INT(Q2=F2)


# ROSA


In [36]:
# 用 df.query 篩選 dataframe 例如 test_data.query('300 <= Days') 
df_rosa = df.query('Customer=="ROSA"')

In [37]:
# 查看 shape 是 Annie 無誤
df_rosa.shape

(30614, 22)

In [38]:
# 取少量來 predict 看看
rosa_100 = df_rosa.sample(100)
%f rosa_100 :> shape -->

rosa_100 :> shape --> (100, 22) (<class 'tuple'>)


In [ ]:
models = project.get_models()

In [39]:
%f project -->
%f models -->
%f cr models :> [0] --> # 這隻的 feature impact 看起來最合理，主要是根據 description 與 subject.

project --> Project(hubble2 component) (<class 'datarobot.models.project.Project'>)
models --> [Model('Keras Slim Residual Neural Network Classifier using Training Schedule (1 Layer: 64 Units)'), Model('AVG Blender'), Model('Regularized Logistic Regression (L2)'), Model('Regularized Logistic Regression (L2)'), Model('Regularized Logistic Regression (L2)'), Model('Regularized Logistic Regression (L2)'), Model('eXtreme Gradient Boosted Trees Classifier'), Model('Light Gradient Boosted Trees Classifier with Early Stopping (SoftMax Loss) (16 leaves)'), Model('RandomForest Classifier (Gini)'), Model('TensorFlow Neural Network Classifier'), Model('ExtraTrees Classifier (Gini)')] (<class 'list'>)

cr models :> [0] --> Model('Keras Slim Residual Neural Network Classifier using Training Schedule (1 Layer: 64 Units)') (<class 'datarobot.models.model.Model'>)
這隻的 feature impact 看起來最合理，主要是根據 description 與 subject.



In [40]:
model = models[0]
%f model -->

model --> Model('Keras Slim Residual Neural Network Classifier using Training Schedule (1 Layer: 64 Units)') (<class 'datarobot.models.model.Model'>)


Before actually requesting predictions, you should upload the dataset you wish to predict via Project.upload_dataset. Previously uploaded datasets can be seen under Project.get_datasets. When uploading the dataset you can provide the path to a local file, a file object, raw file content, a pandas.DataFrame object, or the url to a publicly available dataset.

In [41]:
dataset = project.upload_dataset(rosa_100)

In [42]:
%f dataset --> # 他主動幫忙取了個名字 'predict.csv' 而且與之前的同名，並列在該 model 的 predict UI 中，靠時間分辨。
%f project :> get_datasets() -->

dataset --> PredictionDataset('predict.csv') (<class 'datarobot.models.prediction_dataset.PredictionDataset'>)
他主動幫忙取了個名字 'predict.csv'

project :> get_datasets() --> [PredictionDataset('predict.csv'), PredictionDataset('predict.csv')] (<class 'list'>)


In [46]:
? pd.read_sql

In [45]:
pms = pd.read_sql(r"c:\Users\8304018\Documents\GitHub\examples-for-data-scientists\Playground\QT PMS\LNBIssue.sql")
pms

TypeError: read_sql() missing 1 required positional argument: 'con'

In [47]:
%f *debug*


paste
6paste
I am a student
who's feet?
.
stella'sver

Error! ver unknown.

version
p e f o r t h    v1.26
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.

exit
